# Azure Machine Learning Engineer
## Project 3 - Capstone Hyperdrive

## Create a workspace
In this step we are making an Azure Workspace and setting up an experiment

In [1]:
# Create a new workspace and define an experiment.

from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
ws.get_details()

# Choose a name for the experiment
experiment_name = 'udacity-project-hyperdrive'
experiment = Experiment(workspace=ws, name = experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-146693
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-146693


## Create an environment
Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [2]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults
  - xgboost

Writing conda_dependencies.yml


In [3]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
myenv = Environment.get(workspace=ws, name="AzureML-Minimal")
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

### Setup Compute
Create a new compute or use an existing one if its present

In [4]:
# Create a compute cluster to provision VM Resources.

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = 'cpu-cluster-01'

#Verify that the culster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = cpu_cluster_name)
    print('Found existing cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

Found existing cluster, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

## Run Details

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment

#experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_config, show_output = True)

RunDetails(hyperdrive_run).show()

In [9]:
# Setup Hyperparameter Tuning with Hyperdrive.

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
import shutil
from azureml.core import ScriptRunConfig

#Define the parameter search space/method
# Specify parameter sampler, in this case we are looking to get defined ranges and pass back to the SKILEARN
# training model.
# we can define RandomParameterSampling, Grid Sampling or Bayesian sampling
# ie. ps = GridParameterSampling.... this would require importing from azureml.train.hyperdrive import GridParameterSampling

ps = RandomParameterSampling({
    '--max_depth' : choice(2,5), #limited for simplicity
    '--learning_rate' : choice (0.1,1) #limited for simplicity
})

# Specify an early termination Policy
# Other options are median policy, have stuck with bandit for simplification
# Bandit policy stops if its less than 10% of best model, starting and interval 5
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#creates a training director.
if "training" not in os.listdir():
    os.mkdir("./training")
    
script_folder = './training'

os.makedirs(script_folder, exist_ok = True)

shutil.copy('./trainurl.py',script_folder)

# Create a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory=script_folder,
                      script='trainurl.py',
                      environment=sklearn_env
)

src.run_config.target = compute_target

hyperdrive_config = HyperDriveConfig(estimator = src,
                                     hyperparameter_sampling = ps,
                                     policy = early_termination_policy,
                                     primary_metric_name = 'Accuracy', #Accuracy for classification
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 50,
                                     max_concurrent_runs = 4)

In [11]:
pip show azureml-core

Name: azureml-core
Version: 1.28.0
Summary: Azure Machine Learning core packages, modules, and classes
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: pytz, ndg-httpsclient, pyopenssl, pathspec, azure-mgmt-authorization, contextlib2, jsonpickle, backports.tempfile, msrestazure, azure-mgmt-containerregistry, adal, azure-mgmt-keyvault, azure-common, azure-mgmt-storage, jmespath, cryptography, python-dateutil, SecretStorage, requests, docker, azure-mgmt-resource, ruamel.yaml, azure-graphrbac, msrest, urllib3, PyJWT
Required-by: azureml-widgets, azureml-train-core, azureml-train-automl-runtime, azureml-train-automl-client, azureml-tensorboard, azureml-telemetry, azureml-sdk, azureml-pipeline-core, azureml-opendatasets, azureml-mlflow, azureml-interpret, azureml-defaults, azureml-datadrift, azur

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment

#experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_config, show_output = True)

RunDetails(hyperdrive_run).show()

AttributeError: 'ScriptRunConfig' object has no attribute '_compute_target'

## Best Model

In [25]:
import joblib
# Get your best run and save the model from that run.

best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_hyperdrive_run_metrics = best_hyperdrive_run.get_metrics()

print("Best Run Metrics :", best_hyperdrive_run.get_metrics())

best_hyperdrive_run.download_file(
    best_hyperdrive_run.get_file_names()[-1],
    output_file_path="./outputs/"
)
best_hyperdrive_model = best_hyperdrive_run.register_model(
    model_name="best_hyperdrive_model",
    model_path="./outputs/best_hyperdrive_model.joblib",
    tags=best_hyperdrive_run.get_metrics()
)

Best Run Metrics : {'Accuracy': 0.8686661889650936, 'Max depth:': 2.0, 'Learning rate:': 1}


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.